In [1]:
import numpy as np
from itertools import combinations_with_replacement, combinations
from scipy.optimize import nnls
from tqdm import tqdm

test = True
if test:
    filename = "10_test.txt"
else:
    filename = "10_input.txt"

with open(filename) as f:
    input = f.read().splitlines()
input = [line.split(" ") for line in input]
machines = []
for line in input:
    machine = {}
    machine["indicators"] = line[0][1:-1]
    machine["buttons"] = [eval(numbers) for numbers in line[1:-1]]
    machine["joltages"] = np.array(line[-1][1:-1].split(","),dtype=int)
    machines.append(machine)

In [2]:
# 1:

def indicators_to_vec(indicators):
    indic_arr = np.array(list(indicators))
    indic_vec = np.where(indic_arr == "#", -1, 1)
    indic_vec.astype(int)
    return indic_vec

def button_to_vec(button, num_lights):
    if type(button) == int:
        button = (button,)
    button_vec = np.ones((num_lights,), dtype=int)
    for but in button:
        button_vec[but] = -1
    return button_vec

def find_least_presses(machine):
    indicators = indicators_to_vec(machine["indicators"])
    buttons = [button_to_vec(button, len(indicators)) for button in machine["buttons"]]
    
    min_presses = 0
    while True:
        min_presses += 1
        presses = combinations_with_replacement(range(len(buttons)), r=min_presses)
        for sequence in presses:
            lights = np.ones((len(indicators),), dtype=int)
            for button in sequence:
                lights *= buttons[button]
            if np.all(lights == indicators):
                return min_presses
            
score_1 = 0
for machine in machines:
    score_1 += find_least_presses(machine)
score_1
                

7

In [3]:
# 2:

# different cases:
more_buttons = 0
more_joltages = 0
equal = 0

for machine in machines:
    num_buttons  = len(machine["buttons"])
    num_joltages = len(machine["joltages"])
    if num_buttons > num_joltages:
        more_buttons += 1
    elif num_buttons < num_joltages:
        more_joltages += 1
    else:
        equal += 1

print(more_buttons, more_joltages, equal)

1 1 1


In [4]:
# need to solve linear system:
# - positive solution
# - integer solution
# - optimal solution (minimal sum)

from scipy.optimize import LinearConstraint, Bounds, milp
def solve_positive_integer(A,b):
    num_variables = A.shape[1]
    constraints = LinearConstraint(A,b,b)
    bounds = Bounds(lb=0, ub=np.inf)
    c = np.ones(num_variables)
    integrality = np.ones(num_variables)
    return milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)

In [5]:
# example:
machine = machines[0]
print("machine:\n", machine)

num_joltages = len(machine["joltages"])
button_matrix = np.array([(-button_to_vec(button, num_joltages)  + 1) // 2 for button in machine["buttons"]])
print("encoded buttons:\n", button_matrix)

solution = solve_positive_integer(button_matrix.T,machine["joltages"])
print("solution:\n", solution.x)

print("check:\n", button_matrix.T @ solution.x)

machine:
 {'indicators': '.##.', 'buttons': [3, (1, 3), 2, (2, 3), (0, 2), (0, 1)], 'joltages': array([3, 5, 4, 7])}
encoded buttons:
 [[0 0 0 1]
 [0 1 0 1]
 [0 0 1 0]
 [0 0 1 1]
 [1 0 1 0]
 [1 1 0 0]]
solution:
 [1. 2. 0. 4. 0. 3.]
check:
 [3. 5. 4. 7.]


In [6]:
score = 0
for machine in machines:
    num_buttons  = len(machine["buttons"])
    num_joltages = len(machine["joltages"])

    if num_buttons < num_joltages:
        for _ in range(num_joltages-num_buttons):
            machine["buttons"].append(())
            
    button_matrix = np.array([(-button_to_vec(button, num_joltages)  + 1) // 2 for button in machine["buttons"]])
    solution = solve_positive_integer(button_matrix.T, machine["joltages"]).x
    num_presses = solution.sum()
    score += int(num_presses)
score

33